In [2]:
import csv
# load the data
movies_file = sc.textFile("dataset/movies.csv")
ratings_file = sc.textFile("dataset/ratings.csv")
tags_file = sc.textFile("dataset/tags.csv")
'''
# extract header
header_movies = movies_file.first() 
header_ratings = ratings_file.first()
header_tags = tags_file.first()

'''

# remove the header, separate the elements for each row
data_movies = movies_file.map(lambda row: next(csv.reader(row.splitlines(), skipinitialspace=True)))
data_ratings = ratings_file.map(lambda row: next(csv.reader(row.splitlines(), skipinitialspace=True)))
data_tags = tags_file.map(lambda row: next(csv.reader(row.splitlines(), skipinitialspace=True)))

#data = rawData.filter(lambda row: row != header).map(lambda row : row.split(",")) 
#data2 = rawData2.filter(lambda row: row != header2).map(lambda row : row.split(",")) 
# put the data in cache

table_movies = spark.createDataFrame(data_movies, ['movieId', 'title','genre'])
table_ratings = spark.createDataFrame(data_ratings, ['userId','movieId','rating', 'timestamp' ])
table_tags = spark.createDataFrame(data_tags, ['userId','movieId','tag','timestamp'])


movies = table_movies.alias('movies')
ratings = table_ratings.alias('ratings')
tags = table_tags.alias('tags')

movies.persist()
ratings.persist()
tags.persist()

#inner_join = movies.join(ratings, movies.id == ratings.id)
#inner_join.show(50)


DataFrame[userId: string, movieId: string, tag: string, timestamp: string]

Print all movie titles starring ‘Daniel Craig’, sorted in an ascending alphabetical
order.

In [6]:
inner_join = movies.join(ratings, movies.movieId == ratings.movieId)
#inner_join.show(50)


[Row(movieId='movieId', title='title', genre='genres'), Row(movieId='1', title='Toy Story (1995)', genre='Adventure|Animation|Children|Comedy|Fantasy'), Row(movieId='2', title='Jumanji (1995)', genre='Adventure|Children|Fantasy'), Row(movieId='3', title='Grumpier Old Men (1995)', genre='Comedy|Romance'), Row(movieId='4', title='Waiting to Exhale (1995)', genre='Comedy|Drama|Romance'), Row(movieId='5', title='Father of the Bride Part II (1995)', genre='Comedy'), Row(movieId='6', title='Heat (1995)', genre='Action|Crime|Thriller'), Row(movieId='7', title='Sabrina (1995)', genre='Comedy|Romance'), Row(movieId='8', title='Tom and Huck (1995)', genre='Adventure|Children'), Row(movieId='9', title='Sudden Death (1995)', genre='Action'), Row(movieId='10', title='GoldenEye (1995)', genre='Action|Adventure|Thriller'), Row(movieId='11', title='American President, The (1995)', genre='Comedy|Drama|Romance'), Row(movieId='12', title='Dracula: Dead and Loving It (1995)', genre='Comedy|Horror'), Row(m